In [13]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet import preprocess_input

# Taille des images attendues par le modèle
IMG_SIZE = (224, 224)

# Fonction de chargement des séquences
def load_all_sequences_for_backtest(base_dir="dataset_auto/sequences"):
    """
    Charge toutes les séquences d'images (5 images par séquence) depuis les sous-dossiers uptrend/downtrend.
    Retourne une liste de séquences sous forme de tableaux numpy.
    """
    all_sequences = []

    for trend_label in ["uptrend", "downtrend"]:
        trend_dir = os.path.join(base_dir, trend_label)
        for seq_folder in sorted(os.listdir(trend_dir)):
            seq_path = os.path.join(trend_dir, seq_folder)
            if os.path.isdir(seq_path):
                sequence = []
                for i in range(5):
                    img_path = os.path.join(seq_path, f"img_{i}.png")
                    if os.path.exists(img_path):
                        img = load_img(img_path, target_size=IMG_SIZE)
                        img = img_to_array(img)
                        img = preprocess_input(img)
                        sequence.append(img)
                if len(sequence) == 5:
                    all_sequences.append(np.array(sequence))
    return all_sequences

# Charger le modèle
model = load_model('models\end2end_model_20250530_203710.keras')

# Charger les séquences depuis les images générées
sequences = load_all_sequences_for_backtest()

# Prédire un signal (0 ou 1) pour chaque séquence
signals = []
for seq in sequences:
    seq_input = np.expand_dims(seq, axis=0)  # (1, 5, 224, 224, 3)
    prediction = model.predict(seq_input, verbose=0)
    signal = int(np.argmax(prediction))
    signals.append(signal)

# Afficher les signaux
print("Signaux prédits :", signals)


Signaux prédits : [1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0]


In [25]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet import preprocess_input

# ==== PARAMÈTRES ====
IMG_SIZE = (224, 224)
SEQ_LEN = 5
MODEL_PATH = "models/end2end_model_20250530_203710.keras"
DATA_PATH = "tesla_ohlcv_365jours.csv"
SEQUENCES_DIR = "dataset_auto\sequences"
DAYS_PER_IMAGE = 20
STRIDE = 5

# ==== CHARGEMENT DES DONNÉES ====
df = pd.read_csv(DATA_PATH, parse_dates=["datetime"])
df.rename(columns={
    "datetime": "Date",
    "open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"
}, inplace=True)
df.sort_values("Date", inplace=True)
df.reset_index(drop=True, inplace=True)

# ==== CHARGEMENT DU MODÈLE ====
print("📦 Chargement du modèle...")
model = load_model(MODEL_PATH)

# ==== PRÉPARATION DES SÉQUENCES ====
print("🧩 Chargement des séquences...")
sequences = []
sequence_dates = []

for trend_label in ["uptrend", "downtrend"]:
    trend_dir = os.path.join(SEQUENCES_DIR, trend_label)
    if not os.path.exists(trend_dir):
        continue

    for seq_folder in sorted(os.listdir(trend_dir), key=lambda x: int(x.split("_")[1])):
        seq_path = os.path.join(trend_dir, seq_folder)
        if not os.path.isdir(seq_path):
            continue

        sequence = []
        for i in range(SEQ_LEN):
            img_path = os.path.join(seq_path, f"img_{i}.png")
            if os.path.exists(img_path):
                img = load_img(img_path, target_size=IMG_SIZE)
                img = img_to_array(img)
                img = preprocess_input(img)
                sequence.append(img)

        if len(sequence) == SEQ_LEN:
            sequences.append(np.array(sequence))

            # Date correspondant à la fin de la séquence
            seq_index = int(seq_folder.split("_")[1])
            last_candle_index = seq_index * STRIDE + (SEQ_LEN - 1) * DAYS_PER_IMAGE
            if last_candle_index < len(df):
                sequence_dates.append(df.iloc[last_candle_index]["Date"])

# ==== PRÉDICTIONS ====
print("📈 Prédiction des signaux...")
signals = []
for seq in sequences:
    seq_input = np.expand_dims(seq, axis=0)  # Shape: (1, 5, 224, 224, 3)
    prediction = model.predict(seq_input, verbose=0)
    signal = int(np.argmax(prediction))  # 0 = Buy, 1 = Sell
    signals.append(signal)

# ==== AFFECTATION DES SIGNAUX AU DATAFRAME ====
df["Signal"] = -1
for date, sig in zip(sequence_dates, signals):
    df.loc[df["Date"] == date, "Signal"] = sig

# ==== AFFICHAGE DES PRÉDICTIONS ====
print("\n📋 === DÉTAILS DES PRÉDICTIONS ===")
for date, sig in zip(sequence_dates, signals):
    close_price = df[df["Date"] == date]["Close"].values[0]
    signal_str = "Buy" if sig == 0 else "Sell"
    print(f"{signal_str} @ {close_price:.2f} € le {date.date()}")

# ==== VISUALISATION AVEC PLOTLY ====
fig = go.Figure(data=[go.Candlestick(
    x=df["Date"],
    open=df["Open"], high=df["High"],
    low=df["Low"], close=df["Close"],
    name="Prix"
)])

# Ajout des signaux
colors = {0: "green", 1: "red", -1: "gray"}
symbols = {0: "triangle-up", 1: "triangle-down", -1: "circle"}

for sig_value in [0, 1]:
    subset = df[df["Signal"] == sig_value]
    fig.add_trace(go.Scatter(
        x=subset["Date"],
        y=subset["Close"],
        mode="markers",
        name="Buy" if sig_value == 0 else "Sell",
        marker=dict(color=colors[sig_value], size=8, symbol=symbols[sig_value])
    ))

# Dernier signal personnalisé
if len(signals) > 0:
    last_date = sequence_dates[-1]
    last_row = df[df["Date"] == last_date].iloc[0]
    signal_value = int(last_row["Signal"])
    signal_type = {0: "Buy", 1: "Sell", -1: "None"}[signal_value]

    fig.add_trace(go.Scatter(
        x=[last_row["Date"]],
        y=[last_row["Close"]],
        mode="markers+text",
        name="Dernier Signal",
        marker=dict(color="blue", size=20, symbol="star-diamond"),
        text=[f"{signal_type}\n{last_row['Close']:.2f} €\n({last_date.date()})"],
        textposition="top center",
        textfont=dict(size=14, color="blue")
    ))

fig.update_layout(
    title="📊 Signaux de Tendance Prédits",
    xaxis_title="Date",
    yaxis_title="Prix",
    xaxis_rangeslider_visible=False,
    legend_title="Signaux"
)
fig.show()

# ==== BACKTEST SIMPLIFIÉ ====
capital = 10000
position = 0
entry_price = 0
trades = []

print("\n📉 === BACKTEST ===")
for i, row in df.iterrows():
    if row["Signal"] == 0 and position == 0:
        entry_price = row["Close"]
        position = capital / entry_price
        trades.append(f"🟢 Achat à {entry_price:.2f} € le {row['Date'].date()}")
    elif row["Signal"] == 1 and position > 0:
        exit_price = row["Close"]
        capital = position * exit_price
        trades.append(f"🔴 Vente à {exit_price:.2f} € le {row['Date'].date()} | 💰 Capital: {capital:.2f} €")
        position = 0

# Vente finale si position encore ouverte
if position > 0:
    final_price = df.iloc[-1]["Close"]
    capital = position * final_price
    trades.append(f"⚪ Vente finale à {final_price:.2f} € le {df.iloc[-1]['Date'].date()} | 🏁 Capital final: {capital:.2f} €")

# Affichage final
print("\n📌 === RÉSUMÉ DES TRADES ===")
for t in trades:
    print(t)
print(f"\n💼 Capital final : {capital:.2f} €")


📦 Chargement du modèle...
🧩 Chargement des séquences...
📈 Prédiction des signaux...

📋 === DÉTAILS DES PRÉDICTIONS ===
Sell @ 149.93 € le 2024-04-18
Sell @ 174.84 € le 2024-05-16
Sell @ 178.01 € le 2024-06-14
Buy @ 182.58 € le 2024-06-24
Buy @ 256.56 € le 2024-07-16
Sell @ 221.10 € le 2024-08-20
Sell @ 219.41 € le 2024-09-04
Buy @ 228.13 € le 2024-09-11
Buy @ 227.20 € le 2024-09-18
Buy @ 257.02 € le 2024-09-25
Sell @ 249.02 € le 2024-10-02
Sell @ 221.33 € le 2024-10-16
Buy @ 257.55 € le 2024-10-30
Sell @ 288.53 € le 2024-11-06
Buy @ 342.03 € le 2024-11-20
Buy @ 332.89 € le 2024-11-27
Buy @ 369.49 € le 2024-12-05
Buy @ 418.10 € le 2024-12-12
Buy @ 411.05 € le 2025-01-06
Buy @ 396.36 € le 2025-01-14
Buy @ 389.10 € le 2025-01-29
Sell @ 336.51 € le 2025-02-12
Sell @ 354.40 € le 2025-02-20
Buy @ 281.95 € le 2025-02-27
Buy @ 263.45 € le 2025-03-06
Sell @ 273.13 € le 2025-03-27
Sell @ 267.28 € le 2025-04-03
Sell @ 241.37 € le 2025-04-17
Sell @ 174.60 € le 2024-04-11
Sell @ 170.18 € le 2024-04


📉 === BACKTEST ===

📌 === RÉSUMÉ DES TRADES ===
🟢 Achat à 171.97 € le 2024-05-09
🔴 Vente à 174.84 € le 2024-05-16 | 💰 Capital: 10166.89 €
🟢 Achat à 182.58 € le 2024-06-24
🔴 Vente à 222.62 € le 2024-07-30 | 💰 Capital: 12396.50 €
🟢 Achat à 207.83 € le 2024-08-13
🔴 Vente à 221.10 € le 2024-08-20 | 💰 Capital: 13188.02 €
🟢 Achat à 228.13 € le 2024-09-11
🔴 Vente à 249.02 € le 2024-10-02 | 💰 Capital: 14395.66 €
🟢 Achat à 213.65 € le 2024-10-23
🔴 Vente à 288.53 € le 2024-11-06 | 💰 Capital: 19441.04 €
🟢 Achat à 342.03 € le 2024-11-20
🔴 Vente à 431.66 € le 2024-12-27 | 💰 Capital: 24535.63 €
🟢 Achat à 411.05 € le 2025-01-06
🔴 Vente à 415.11 € le 2025-01-22 | 💰 Capital: 24777.97 €
🟢 Achat à 389.10 € le 2025-01-29
🔴 Vente à 378.17 € le 2025-02-05 | 💰 Capital: 24081.94 €
🟢 Achat à 281.95 € le 2025-02-27
🔴 Vente à 240.68 € le 2025-03-13 | 💰 Capital: 20556.98 €
🟢 Achat à 252.40 € le 2025-04-10
🔴 Vente à 241.37 € le 2025-04-17 | 💰 Capital: 19658.64 €
🟢 Achat à 284.95 € le 2025-04-25
🔴 Vente à 349.98 €

In [43]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet import preprocess_input
from backtesting import Backtest, Strategy
import warnings
warnings.filterwarnings('ignore')

# === CONFIGURATION ===
class Config:
    IMG_SIZE = (224, 224)
    SEQ_LEN = 5
    MODEL_PATH = "models/end2end_model_20250530_203710.keras"
    DATA_PATH = "tesla_ohlcv_365jours.csv"
    SEQUENCES_DIR = "dataset_auto/sequences"
    DAYS_PER_IMAGE = 20
    STRIDE = 5
    INITIAL_CASH = 10000
    COMMISSION = 0.002
    SLIPPAGE = 0.001
    STOP_LOSS = 0.10
    TAKE_PROFIT = 0.20

# === INDICATEURS TECHNIQUES ===
def SMA(arr, n):
    return pd.Series(arr).rolling(n).mean().values

# === STRATÉGIE AMÉLIORÉE ===
class MLStrategy(Strategy):
    stop_loss_pct = Config.STOP_LOSS
    take_profit_pct = Config.TAKE_PROFIT
    
    def init(self):
        close_prices = np.array(self.data.Close)
        self.sma20 = self.I(SMA, close_prices, 20, name='SMA20')
        self.sma50 = self.I(SMA, close_prices, 50, name='SMA50')
        self.signal = np.array(self.data.Signal)
        self._equity_history = []
    
    def next(self):
        current_idx = len(self.data.Close) - 1
        long_cond = (self.signal[current_idx] == 0) and (self.sma20[current_idx] > self.sma50[current_idx])
        short_cond = (self.signal[current_idx] == 1) and (self.sma20[current_idx] < self.sma50[current_idx])
        
        if not self.position:
            price = self.data.Close[-1]
            if long_cond:
                self.buy(sl=price*(1-self.stop_loss_pct), tp=price*(1+self.take_profit_pct))
            elif short_cond:
                self.sell(sl=price*(1+self.stop_loss_pct), tp=price*(1-self.take_profit_pct))
        else:
            if self.position.is_long and self.signal[current_idx] == 1:
                self.position.close()
            elif self.position.is_short and self.signal[current_idx] == 0:
                self.position.close()
        
        self._equity_history.append(self.equity)

# === AFFICHAGE DES RÉSULTATS ===
def display_backtest_results(results):
    # Conversion des résultats en DataFrame
    res_df = pd.DataFrame({
        'Metric': [
            'Start', 'End', 'Duration', 
            'Exposure Time [%]', 'Equity Final [$]', 
            'Equity Peak [$]', 'Return [%]',
            'Buy & Hold Return [%]', 'Return (Ann.) [%]',
            'Volatility (Ann.) [%]', 'CAGR [%]',
            'Sharpe Ratio', 'Sortino Ratio',
            'Max. Drawdown [%]', '# Trades',
            'Win Rate [%]', 'Profit Factor'
        ],
        'Value': [
            results['Start'],
            results['End'],
            str(results['Duration']),
            f"{results['Exposure Time [%]']:.2f}%",
            f"${results['Equity Final [$]']:,.2f}",
            f"${results['Equity Peak [$]']:,.2f}",
            f"{results['Return [%]']:.2f}%",
            f"{results['Buy & Hold Return [%]']:.2f}%",
            f"{results['Return (Ann.) [%]']:.2f}%",
            f"{results['Volatility (Ann.) [%]']:.2f}%",
            f"{results['CAGR [%]']:.2f}%",
            f"{results['Sharpe Ratio']:.2f}" if not np.isnan(results['Sharpe Ratio']) else 'NaN',
            f"{results['Sortino Ratio']:.2f}" if not np.isnan(results['Sortino Ratio']) else 'NaN',
            f"{results['Max. Drawdown [%]']:.2f}%",
            results['# Trades'],
            f"{results['Win Rate [%]']:.2f}%" if not np.isnan(results['Win Rate [%]']) else 'NaN',
            f"{results['Profit Factor']:.2f}" if not np.isnan(results['Profit Factor']) else 'NaN'
        ]
    })
    
    fig = go.Figure(data=[go.Table(
        header=dict(
            values=['<b>Métrique</b>', '<b>Valeur</b>'],
            fill_color='navy',
            font=dict(color='white', size=12),
            align='left'
        ),
        cells=dict(
            values=[res_df['Metric'], res_df['Value']],
            fill_color=[['white','lightgrey']*len(res_df)],
            align='left',
            font=dict(size=11)
        )
    )])
    
    fig.update_layout(
        title='<b>Résultats du Backtesting</b>',
        margin=dict(l=10, r=10, t=60, b=10),
        title_font=dict(size=16, color='darkblue')
    )
    
    fig.show()

# === VISUALISATION DES GRAPHIQUES ===
def plot_trading_results(df, equity_curve):
    # Création des subplots uniquement pour les graphiques
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        row_heights=[0.7, 0.3],
        subplot_titles=("Graphique des Prix", "Courbe d'Équité")
    )
    
    # Graphique des prix (row 1)
    fig.add_trace(go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='OHLC'),
        row=1, col=1
    )
    
    # Indicateurs et signaux
    fig.add_trace(go.Scatter(
        x=df.index, y=df['SMA20'], 
        name='SMA 20', line=dict(color='blue', width=1)),
        row=1, col=1
    )
    
    fig.add_trace(go.Scatter(
        x=df.index, y=df['SMA50'], 
        name='SMA 50', line=dict(color='orange', width=1)),
        row=1, col=1
    )
    
    buy_signals = df[df['Signal'] == 0]
    sell_signals = df[df['Signal'] == 1]
    
    fig.add_trace(go.Scatter(
        x=buy_signals.index, y=buy_signals['Close'],
        mode='markers', name='Buy',
        marker=dict(color='green', size=10, symbol='triangle-up')),
        row=1, col=1
    )
    
    fig.add_trace(go.Scatter(
        x=sell_signals.index, y=sell_signals['Close'],
        mode='markers', name='Sell',
        marker=dict(color='red', size=10, symbol='triangle-down')),
        row=1, col=1
    )
    
    # Courbe d'équité (row 2)
    fig.add_trace(go.Scatter(
        x=equity_curve.index, y=equity_curve['Equity'],
        name='Equity', line=dict(color='purple', width=2)),
        row=2, col=1
    )
    
    # Mise en forme
    fig.update_layout(
        title='Résultats du Trading',
        height=800,
        showlegend=True,
        xaxis_rangeslider_visible=False,
        hovermode='x unified'
    )
    
    fig.update_yaxes(title_text="Prix ($)", row=1, col=1)
    fig.update_yaxes(title_text="Capital ($)", row=2, col=1)
    
    fig.show()


# === BACKTEST COMPLET ===
def run_backtest():
    # 1. Chargement des données
    print("📊 Chargement des données...")
    df = pd.read_csv(Config.DATA_PATH, parse_dates=['datetime'])
    df.rename(columns={
        'datetime':'Date', 'open':'Open', 'high':'High',
        'low':'Low', 'close':'Close', 'volume':'Volume'
    }, inplace=True)
    
    # 2. Génération des signaux
    print("🔍 Génération des signaux...")
    df['Signal'] = -1
    for i in range(len(df)):
        if i % 10 == 0: df.at[i, 'Signal'] = 0
        elif i % 7 == 0: df.at[i, 'Signal'] = 1
    
    # 3. Backtesting
    print("\n🧪 Lancement du backtest avec bibliothèque 'backtesting'...")
    bt_data = df[['Date','Open','High','Low','Close','Volume','Signal']].copy()
    bt_data.set_index('Date', inplace=True)
    
    bt = Backtest(bt_data, MLStrategy,
                 cash=Config.INITIAL_CASH,
                 commission=Config.COMMISSION)
    
    results = bt.run()
    
    # 4. Affichage des résultats
    print("\n📊 === RÉSULTATS DÉTAILLÉS ===")
    display_backtest_results(results)
    
    # 5. Préparation des données pour visualisation
    df['SMA20'] = SMA(df['Close'], 20)
    df['SMA50'] = SMA(df['Close'], 50)
    df.set_index('Date', inplace=True)
    
    equity = pd.DataFrame({'Equity': results['_equity_curve']['Equity']})
    
    # 6. Affichage console
    print("\n💡 === SYNTHÈSE ===")
    print(f"Return: {results['Return [%]']:.2f}%")
    print(f"Sharpe Ratio: {results['Sharpe Ratio']:.2f}" if not np.isnan(results['Sharpe Ratio']) else 'Sharpe Ratio: NaN')
    print(f"Max Drawdown: {results['Max. Drawdown [%]']:.2f}%")
    print(f"Nombre de trades: {results['# Trades']}")
    plot_trading_results(df, equity)  # Graphiques ensuite

if __name__ == '__main__':
    run_backtest()

📊 Chargement des données...
🔍 Génération des signaux...

🧪 Lancement du backtest avec bibliothèque 'backtesting'...



📊 === RÉSULTATS DÉTAILLÉS ===



💡 === SYNTHÈSE ===
Return: -5.45%
Sharpe Ratio: -0.09
Max Drawdown: -36.64%
Nombre de trades: 31


In [67]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet import preprocess_input
from backtesting import Backtest, Strategy
import warnings
warnings.filterwarnings('ignore')

# === CONFIGURATION ===
class Config:
    IMG_SIZE = (224, 224)
    SEQ_LEN = 5
    MODEL_PATH = "models/end2end_model_20250530_203710.keras"
    DATA_PATH = "tesla_ohlcv_365jours.csv"
    SEQUENCES_DIR = "dataset_auto/sequences"
    DAYS_PER_IMAGE = 20
    STRIDE = 5
    INITIAL_CASH = 10000
    COMMISSION = 0.002
    SLIPPAGE = 0.001
    STOP_LOSS = 0.10
    TAKE_PROFIT = 0.20

# === FONCTION POUR GÉNÉRER LES SIGNAUX ===
def generate_signals(df):
    """Génère les signaux à partir du modèle de deep learning"""
    print("🔮 Chargement du modèle et génération des signaux...")
    
    try:
        model = load_model(Config.MODEL_PATH)
        df['Signal'] = -1  # Initialisation des signaux
        
        for trend_dir in ['uptrend', 'downtrend']:
            dir_path = os.path.join(Config.SEQUENCES_DIR, trend_dir)
            if not os.path.exists(dir_path):
                continue
                
            for seq_folder in sorted(os.listdir(dir_path), key=lambda x: int(x.split('_')[1])):
                seq_path = os.path.join(dir_path, seq_folder)
                if not os.path.isdir(seq_path):
                    continue
                
                # Chargement des images
                sequence = []
                valid_sequence = True
                for i in range(Config.SEQ_LEN):
                    img_path = os.path.join(seq_path, f'img_{i}.png')
                    if not os.path.exists(img_path):
                        valid_sequence = False
                        break
                        
                    try:
                        img = load_img(img_path, target_size=Config.IMG_SIZE)
                        img = img_to_array(img)
                        img = preprocess_input(img)
                        sequence.append(img)
                    except Exception as e:
                        print(f"Erreur traitement image {img_path}: {str(e)}")
                        valid_sequence = False
                        break
                
                if valid_sequence and len(sequence) == Config.SEQ_LEN:
                    # Prédiction du modèle
                    seq_array = np.expand_dims(np.array(sequence), axis=0)
                    try:
                        prediction = model.predict(seq_array, verbose=0)
                        signal = np.argmax(prediction)
                        
                        # Correspondance avec les données financières
                        seq_idx = int(seq_folder.split('_')[1])
                        candle_idx = seq_idx * Config.STRIDE + (Config.SEQ_LEN - 1) * Config.DAYS_PER_IMAGE
                        if candle_idx < len(df):
                            df.iloc[candle_idx, df.columns.get_loc('Signal')] = signal
                    except Exception as e:
                        print(f"Erreur prédiction modèle: {str(e)}")
    
    except Exception as e:
        print(f"Erreur critique dans generate_signals: {str(e)}")
        raise
    
    return df

# === INDICATEURS TECHNIQUES ===
def SMA(arr, n):
    return pd.Series(arr).rolling(n).mean().values

# === STRATÉGIE AMÉLIORÉE ===
class MLStrategy(Strategy):
    stop_loss_pct = Config.STOP_LOSS
    take_profit_pct = Config.TAKE_PROFIT
    
    def init(self):
        close_prices = np.array(self.data.Close)
        self.sma20 = self.I(SMA, close_prices, 20, name='SMA20')
        self.sma50 = self.I(SMA, close_prices, 50, name='SMA50')
        self.signal = np.array(self.data.Signal)
        self._equity_history = []
    
    def next(self):
        current_idx = len(self.data.Close) - 1
        
        # Conditions de trading avec filtres SMA
        long_cond = (self.signal[current_idx] == 0) and (self.sma20[current_idx] > self.sma50[current_idx])
        short_cond = (self.signal[current_idx] == 1) and (self.sma20[current_idx] < self.sma50[current_idx])
        
        if not self.position:
            price = self.data.Close[-1]
            if long_cond:
                self.buy(sl=price*(1-self.stop_loss_pct), tp=price*(1+self.take_profit_pct))
            elif short_cond:
                self.sell(sl=price*(1+self.stop_loss_pct), tp=price*(1-self.take_profit_pct))
        else:
            # Fermeture de position si signal contraire
            if self.position.is_long and self.signal[current_idx] == 1:
                self.position.close()
            elif self.position.is_short and self.signal[current_idx] == 0:
                self.position.close()
        
        self._equity_history.append(self.equity)

# === AFFICHAGE DES RÉSULTATS ===
def display_backtest_results(results):
    metrics = {
        'Return [%]': f"{results['Return [%]']:.2f}%",
        'Annual Return [%]': f"{results['Return (Ann.) [%]']:.2f}%",
        'Sharpe Ratio': f"{results['Sharpe Ratio']:.2f}" if not np.isnan(results['Sharpe Ratio']) else 'NaN',
        'Max Drawdown [%]': f"{results['Max. Drawdown [%]']:.2f}%",
        'Win Rate [%]': f"{results['Win Rate [%]']:.2f}%" if not np.isnan(results['Win Rate [%]']) else 'NaN',
        'Profit Factor': f"{results['Profit Factor']:.2f}" if not np.isnan(results['Profit Factor']) else 'NaN',
        'Trades': results['# Trades']
    }
    
    print("\n📊 === PERFORMANCE SUMMARY ===")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

# === VISUALISATION DES GRAPHIQUES ===
def plot_trading_results(df, equity_curve):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                       vertical_spacing=0.05,
                       row_heights=[0.7, 0.3],
                       subplot_titles=("Price Chart", "Equity Curve"))
    
    # Price Chart
    fig.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'],
                                low=df['Low'], close=df['Close'], name='OHLC'),
                 row=1, col=1)
    
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA20'], name='SMA 20',
                            line=dict(color='blue', width=1)),
                 row=1, col=1)
    
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA50'], name='SMA 50',
                            line=dict(color='orange', width=1)),
                 row=1, col=1)
    
    # Signals
    buy_signals = df[df['Signal'] == 0]
    sell_signals = df[df['Signal'] == 1]
    
    fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'],
                            mode='markers', name='Buy',
                            marker=dict(color='green', size=10, symbol='triangle-up')),
                 row=1, col=1)
    
    fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'],
                            mode='markers', name='Sell',
                            marker=dict(color='red', size=10, symbol='triangle-down')),
                 row=1, col=1)
    
    # Equity Curve
    fig.add_trace(go.Scatter(x=equity_curve.index, y=equity_curve['Equity'],
                            name='Equity', line=dict(color='purple', width=2)),
                 row=2, col=1)
    
    fig.update_layout(height=800, showlegend=True,
                    xaxis_rangeslider_visible=False,
                    hovermode='x unified')
    fig.show()

# === BACKTEST COMPLET ===
def run_backtest():
    try:
        # 1. Chargement des données
        print("📊 Chargement des données...")
        df = pd.read_csv(Config.DATA_PATH, parse_dates=['datetime'])
        
        # Nettoyage des données
        df.rename(columns={
            'datetime': 'Date',
            'open': 'Open', 
            'high': 'High',
            'low': 'Low', 
            'close': 'Close', 
            'volume': 'Volume'
        }, inplace=True)
        
        df.sort_values('Date', inplace=True)
        df.set_index('Date', inplace=True)
        
        # 2. Génération des signaux
        df = generate_signals(df)
        
        # 3. Calcul des indicateurs
        df['SMA20'] = SMA(df['Close'], 20)
        df['SMA50'] = SMA(df['Close'], 50)
        
        # 4. Backtesting
        print("\n🧪 Lancement du backtest...")
        bt = Backtest(df, MLStrategy,
                     cash=Config.INITIAL_CASH,
                     commission=Config.COMMISSION)
        
        results = bt.run()
        
        # 5. Affichage des résultats
        display_backtest_results(results)
        
        # 6. Visualisation
        equity_curve = pd.DataFrame({'Equity': results['_equity_curve']['Equity']})
        plot_trading_results(df, equity_curve)
        
    except Exception as e:
        print(f"\n❌ ERREUR: {str(e)}")
        print("Vérifiez les chemins des fichiers et le format des données")

if __name__ == '__main__':
    run_backtest()

📊 Chargement des données...
🔮 Chargement du modèle et génération des signaux...

🧪 Lancement du backtest...



📊 === PERFORMANCE SUMMARY ===
Return [%]: 53.99%
Annual Return [%]: 34.72%
Sharpe Ratio: 0.65
Max Drawdown [%]: -31.98%
Win Rate [%]: 53.33%
Profit Factor: 2.08
Trades: 15


In [80]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet import preprocess_input
from backtesting import Backtest, Strategy
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# === CONFIGURATION ===
class Config:
    IMG_SIZE = (224, 224)
    SEQ_LEN = 5
    MODEL_PATH = "models/end2end_model_20250530_203710.keras"
    DATA_PATH = "tesla_ohlcv_365jours.csv"
    SEQUENCES_DIR = "dataset_auto/sequences"
    DAYS_PER_IMAGE = 20
    STRIDE = 5
    INITIAL_CASH = 10000
    COMMISSION = 0.002
    SLIPPAGE = 0.001
    STOP_LOSS = 0.10
    TAKE_PROFIT = 0.20
    RISK_FREE_RATE = 0.0

# === FONCTION POUR GÉNÉRER LES SIGNAUX ===
def generate_signals(df):
    """Génère les signaux à partir du modèle de deep learning"""
    print("🔮 Chargement du modèle et génération des signaux...")
    
    try:
        model = load_model(Config.MODEL_PATH)
        df['Signal'] = -1  # -1 = pas de signal, 0 = buy, 1 = sell
        predictions = []
        actuals = []
        prediction_dates = []
        
        # Vérification de l'existence des dossiers
        if not os.path.exists(Config.SEQUENCES_DIR):
            raise FileNotFoundError(f"Le dossier {Config.SEQUENCES_DIR} n'existe pas")
        
        for trend_dir in ['uptrend', 'downtrend']:
            dir_path = os.path.join(Config.SEQUENCES_DIR, trend_dir)
            if not os.path.exists(dir_path):
                print(f"Attention: dossier {dir_path} non trouvé")
                continue
                
            for seq_folder in sorted(os.listdir(dir_path), key=lambda x: int(x.split('_')[1])):
                seq_path = os.path.join(dir_path, seq_folder)
                if not os.path.isdir(seq_path):
                    continue
                
                # Chargement des images
                sequence = []
                valid_sequence = True
                for i in range(Config.SEQ_LEN):
                    img_path = os.path.join(seq_path, f'img_{i}.png')
                    if not os.path.exists(img_path):
                        valid_sequence = False
                        break
                        
                    try:
                        img = load_img(img_path, target_size=Config.IMG_SIZE)
                        img = img_to_array(img)
                        img = preprocess_input(img)
                        sequence.append(img)
                    except Exception as e:
                        print(f"Erreur traitement image {img_path}: {str(e)}")
                        valid_sequence = False
                        break
                
                if valid_sequence and len(sequence) == Config.SEQ_LEN:
                    try:
                        # Prédiction du modèle
                        seq_array = np.expand_dims(np.array(sequence), axis=0)
                        prediction = model.predict(seq_array, verbose=0)
                        signal = np.argmax(prediction)
                        
                        # Correspondance avec les données financières
                        seq_idx = int(seq_folder.split('_')[1])
                        candle_idx = seq_idx * Config.STRIDE + (Config.SEQ_LEN - 1) * Config.DAYS_PER_IMAGE
                        
                        if candle_idx < len(df):
                            df.iloc[candle_idx, df.columns.get_loc('Signal')] = signal
                            
                            # Stocker pour évaluation
                            predictions.append(signal)
                            actual = 0 if df.iloc[candle_idx]['Close'] > df.iloc[candle_idx-1]['Close'] else 1
                            actuals.append(actual)
                            prediction_dates.append(df.index[candle_idx])
                        else:
                            print(f"Index {candle_idx} hors limites pour la séquence {seq_folder}")
                    except Exception as e:
                        print(f"Erreur lors de la prédiction pour {seq_folder}: {str(e)}")
    
    except Exception as e:
        print(f"Erreur critique dans generate_signals: {str(e)}")
        raise
    
    # Vérification qu'on a bien généré des signaux
    if len(predictions) == 0:
        raise ValueError("Aucun signal généré - vérifiez vos données et chemins d'accès")
    
    return df, predictions, actuals, prediction_dates

# === STRATÉGIE AMÉLIORÉE ===
class MLStrategy(Strategy):
    stop_loss_pct = Config.STOP_LOSS
    take_profit_pct = Config.TAKE_PROFIT
    
    def init(self):
        close_prices = np.array(self.data.Close)
        self.sma20 = self.I(SMA, close_prices, 20, name='SMA20')
        self.sma50 = self.I(SMA, close_prices, 50, name='SMA50')
        self.signal = np.array(self.data.Signal)
        
        # Vérification des signaux
        if len(np.unique(self.signal)) == 1 and self.signal[0] == -1:
            raise ValueError("Aucun signal valide trouvé - vérifiez la génération des signaux")
    
    def next(self):
        current_idx = len(self.data.Close) - 1
        
        # Conditions de trading avec filtres SMA
        long_cond = (self.signal[current_idx] == 0) and (self.sma20[current_idx] > self.sma50[current_idx])
        short_cond = (self.signal[current_idx] == 1) and (self.sma20[current_idx] < self.sma50[current_idx])
        
        if not self.position:
            price = self.data.Close[-1]
            if long_cond:
                self.buy(sl=price*(1-self.stop_loss_pct), tp=price*(1+self.take_profit_pct))
            elif short_cond:
                self.sell(sl=price*(1+self.stop_loss_pct), tp=price*(1-self.take_profit_pct))
        else:
            if self.position.is_long and self.signal[current_idx] == 1:
                self.position.close()
            elif self.position.is_short and self.signal[current_idx] == 0:
                self.position.close()

# === BACKTEST COMPLET ===
def run_backtest():
    try:
        print("📊 Chargement des données...")
        # Chargement avec vérification
        if not os.path.exists(Config.DATA_PATH):
            raise FileNotFoundError(f"Fichier {Config.DATA_PATH} introuvable")
        
        df = pd.read_csv(Config.DATA_PATH, parse_dates=['datetime'])
        if df.empty:
            raise ValueError("Le fichier de données est vide")
        
        # Nettoyage des données
        required_cols = ['datetime', 'open', 'high', 'low', 'close', 'volume']
        if not all(col in df.columns for col in required_cols):
            raise ValueError("Colonnes manquantes dans les données")
        
        df.rename(columns={
            'datetime': 'Date',
            'open': 'Open', 
            'high': 'High',
            'low': 'Low', 
            'close': 'Close', 
            'volume': 'Volume'
        }, inplace=True)
        
        df.sort_values('Date', inplace=True)
        df.set_index('Date', inplace=True)
        
        # Vérification de la plage de dates
        if len(df) < Config.SEQ_LEN * Config.DAYS_PER_IMAGE:
            raise ValueError("Données insuffisantes pour la longueur de séquence configurée")
        
        # Génération des signaux
        df, predictions, actuals, prediction_dates = generate_signals(df)
        
        # Calcul des indicateurs
        df['SMA20'] = SMA(df['Close'], 20)
        df['SMA50'] = SMA(df['Close'], 50)
        
        print("\n🧪 Lancement du backtest...")
        bt = Backtest(df, MLStrategy,
                     cash=Config.INITIAL_CASH,
                     commission=Config.COMMISSION)
        
        results = bt.run()
        
        # Affichage des résultats
        print("\n✅ Backtest terminé avec succès!")
        print(f"Return final: {(results['Equity Final [$]'] - Config.INITIAL_CASH)/Config.INITIAL_CASH*100:.2f}%")
        print(f"Nombre de trades: {len(results['_trades'])}")
        
        # Visualisation
        equity_curve = pd.DataFrame({'Equity': results['_equity_curve']['Equity']})
        plot_results(df, equity_curve, predictions, actuals, prediction_dates)
        
    except Exception as e:
        print(f"\n❌ ERREUR CRITIQUE: {str(e)}")
        print("Problèmes possibles:")
        print("- Chemins d'accès incorrects aux fichiers")
        print("- Données manquantes ou corrompues")
        print("- Modèle incompatible")
        print("- Paramètres de séquence inadaptés à vos données")

if __name__ == '__main__':
    run_backtest()

📊 Chargement des données...
🔮 Chargement du modèle et génération des signaux...
Index 365 hors limites pour la séquence seq_57
Index 370 hors limites pour la séquence seq_58
Index 385 hors limites pour la séquence seq_61
Index 395 hors limites pour la séquence seq_63
Index 405 hors limites pour la séquence seq_65
Index 415 hors limites pour la séquence seq_67
Index 375 hors limites pour la séquence seq_59
Index 380 hors limites pour la séquence seq_60
Index 390 hors limites pour la séquence seq_62
Index 400 hors limites pour la séquence seq_64
Index 410 hors limites pour la séquence seq_66

🧪 Lancement du backtest...



✅ Backtest terminé avec succès!
Return final: 53.99%
Nombre de trades: 15


In [82]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet import preprocess_input
from backtesting import Backtest, Strategy
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# === CONFIGURATION ===
class Config:
    IMG_SIZE = (224, 224)
    SEQ_LEN = 5
    MODEL_PATH = "models/end2end_model_20250530_203710.keras"
    DATA_PATH = "tesla_ohlcv_365jours.csv"
    SEQUENCES_DIR = "dataset_auto/sequences"
    DAYS_PER_IMAGE = 20
    STRIDE = 5
    INITIAL_CASH = 10000
    COMMISSION = 0.002
    SLIPPAGE = 0.001
    STOP_LOSS = 0.10
    TAKE_PROFIT = 0.20
    RISK_FREE_RATE = 0.0

# === INDICATEUR SMA ===
def SMA(arr, n):
    return pd.Series(arr).rolling(n).mean().values

# === GÉNÉRATION DES SIGNAUX ===
def generate_signals(df):
    print("🔮 Chargement du modèle et génération des signaux...")
    
    try:
        model = load_model(Config.MODEL_PATH)
        df['Signal'] = -1
        predictions, actuals, prediction_dates = [], [], []
        
        if not os.path.exists(Config.SEQUENCES_DIR):
            raise FileNotFoundError(f"Le dossier {Config.SEQUENCES_DIR} n'existe pas")
        
        for trend_dir in ['uptrend', 'downtrend']:
            dir_path = os.path.join(Config.SEQUENCES_DIR, trend_dir)
            if not os.path.exists(dir_path):
                print(f"Attention: dossier {dir_path} non trouvé")
                continue
            
            for seq_folder in sorted(os.listdir(dir_path), key=lambda x: int(x.split('_')[1])):
                seq_path = os.path.join(dir_path, seq_folder)
                if not os.path.isdir(seq_path):
                    continue
                
                sequence = []
                valid_sequence = True
                for i in range(Config.SEQ_LEN):
                    img_path = os.path.join(seq_path, f'img_{i}.png')
                    if not os.path.exists(img_path):
                        valid_sequence = False
                        break
                    try:
                        img = load_img(img_path, target_size=Config.IMG_SIZE)
                        img = img_to_array(img)
                        img = preprocess_input(img)
                        sequence.append(img)
                    except Exception as e:
                        print(f"Erreur traitement image {img_path}: {str(e)}")
                        valid_sequence = False
                        break
                
                if valid_sequence and len(sequence) == Config.SEQ_LEN:
                    try:
                        seq_array = np.expand_dims(np.array(sequence), axis=0)
                        prediction = model.predict(seq_array, verbose=0)
                        signal = np.argmax(prediction)
                        
                        seq_idx = int(seq_folder.split('_')[1])
                        candle_idx = seq_idx * Config.STRIDE + (Config.SEQ_LEN - 1) * Config.DAYS_PER_IMAGE
                        
                        if candle_idx >= len(df):
                            print(f"Index {candle_idx} hors limites pour la séquence {seq_folder} (max = {len(df)-1})")
                            continue
                        
                        df.iloc[candle_idx, df.columns.get_loc('Signal')] = signal
                        predictions.append(signal)
                        actual = 0 if df.iloc[candle_idx]['Close'] > df.iloc[candle_idx-1]['Close'] else 1
                        actuals.append(actual)
                        prediction_dates.append(df.index[candle_idx])
                    except Exception as e:
                        print(f"Erreur lors de la prédiction pour {seq_folder}: {str(e)}")
    except Exception as e:
        print(f"Erreur critique dans generate_signals: {str(e)}")
        raise
    
    if len(predictions) == 0:
        raise ValueError("Aucun signal généré - vérifiez vos données et chemins d'accès")
    
    return df, predictions, actuals, prediction_dates

# === STRATÉGIE ML ===
class MLStrategy(Strategy):
    stop_loss_pct = Config.STOP_LOSS
    take_profit_pct = Config.TAKE_PROFIT
    
    def init(self):
        close_prices = np.array(self.data.Close)
        self.sma20 = self.I(SMA, close_prices, 20, name='SMA20')
        self.sma50 = self.I(SMA, close_prices, 50, name='SMA50')
        self.signal = np.array(self.data.Signal)
        if len(np.unique(self.signal)) == 1 and self.signal[0] == -1:
            raise ValueError("Aucun signal valide trouvé - vérifiez la génération des signaux")
    
    def next(self):
        current_idx = len(self.data.Close) - 1
        long_cond = (self.signal[current_idx] == 0) and (self.sma20[current_idx] > self.sma50[current_idx])
        short_cond = (self.signal[current_idx] == 1) and (self.sma20[current_idx] < self.sma50[current_idx])
        
        if not self.position:
            price = self.data.Close[-1]
            if long_cond:
                self.buy(sl=price*(1-self.stop_loss_pct), tp=price*(1+self.take_profit_pct))
            elif short_cond:
                self.sell(sl=price*(1+self.stop_loss_pct), tp=price*(1-self.take_profit_pct))
        else:
            if self.position.is_long and self.signal[current_idx] == 1:
                self.position.close()
            elif self.position.is_short and self.signal[current_idx] == 0:
                self.position.close()

# === VISUALISATION ===
def plot_trading_results(df, equity_curve):
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05,
        row_heights=[0.7, 0.3], subplot_titles=("Graphique des Prix", "Courbe d'Équité")
    )
    
    fig.add_trace(go.Candlestick(
        x=df.index, open=df['Open'], high=df['High'],
        low=df['Low'], close=df['Close'], name='OHLC'),
        row=1, col=1
    )
    
    fig.add_trace(go.Scatter(
        x=df.index, y=df['SMA20'], name='SMA 20',
        line=dict(color='blue', width=1)), row=1, col=1
    )
    fig.add_trace(go.Scatter(
        x=df.index, y=df['SMA50'], name='SMA 50',
        line=dict(color='orange', width=1)), row=1, col=1
    )
    
    buy_signals = df[df['Signal'] == 0]
    sell_signals = df[df['Signal'] == 1]
    fig.add_trace(go.Scatter(
        x=buy_signals.index, y=buy_signals['Close'],
        mode='markers', name='Buy', marker=dict(color='green', size=10, symbol='triangle-up')),
        row=1, col=1
    )
    fig.add_trace(go.Scatter(
        x=sell_signals.index, y=sell_signals['Close'],
        mode='markers', name='Sell', marker=dict(color='red', size=10, symbol='triangle-down')),
        row=1, col=1
    )
    
    fig.add_trace(go.Scatter(
        x=equity_curve.index, y=equity_curve['Equity'],
        name='Equity', line=dict(color='purple', width=2)),
        row=2, col=1
    )
    
    fig.update_layout(
        title='Résultats du Trading',
        height=800,
        showlegend=True,
        xaxis_rangeslider_visible=False,
        hovermode='x unified'
    )
    fig.update_yaxes(title_text="Prix ($)", row=1, col=1)
    fig.update_yaxes(title_text="Capital ($)", row=2, col=1)
    fig.show()

# === BACKTEST PRINCIPAL ===
def run_backtest():
    try:
        print("📊 Chargement des données...")
        if not os.path.exists(Config.DATA_PATH):
            raise FileNotFoundError(f"Fichier {Config.DATA_PATH} introuvable")
        
        df = pd.read_csv(Config.DATA_PATH, parse_dates=['datetime'])
        if df.empty:
            raise ValueError("Le fichier de données est vide")
        
        df.rename(columns={
            'datetime': 'Date', 'open': 'Open', 'high': 'High',
            'low': 'Low', 'close': 'Close', 'volume': 'Volume'
        }, inplace=True)
        df.sort_values('Date', inplace=True)
        df.set_index('Date', inplace=True)
        
        if len(df) < Config.SEQ_LEN * Config.DAYS_PER_IMAGE:
            raise ValueError("Données insuffisantes pour la longueur de séquence configurée")
        
        df, predictions, actuals, prediction_dates = generate_signals(df)
        df['SMA20'] = SMA(df['Close'], 20)
        df['SMA50'] = SMA(df['Close'], 50)
        
        print("\n🧪 Lancement du backtest...")
        bt = Backtest(df, MLStrategy, cash=Config.INITIAL_CASH, commission=Config.COMMISSION)
        results = bt.run()
        
        print("\n✅ Backtest terminé avec succès!")
        print(f"Return final: {(results['Equity Final [$]'] - Config.INITIAL_CASH)/Config.INITIAL_CASH*100:.2f}%")
        print(f"Nombre de trades: {results['# Trades']}")
        
        trades_df = results['_trades']
        num_trades = len(trades_df)
        num_winning = len(trades_df[trades_df['PnL'] > 0])
        num_losing = len(trades_df[trades_df['PnL'] < 0])
        win_rate = (num_winning / num_trades) * 100 if num_trades else 0
        profit_factor = trades_df['PnL'][trades_df['PnL'] > 0].sum() / abs(trades_df['PnL'][trades_df['PnL'] < 0].sum()) if num_losing else float('inf')
        avg_gain = trades_df['PnL'][trades_df['PnL'] > 0].mean() if num_winning else 0
        avg_loss = trades_df['PnL'][trades_df['PnL'] < 0].mean() if num_losing else 0
        avg_duration = trades_df['Duration'].mean()
        avg_duration_days = avg_duration.total_seconds() / (24 * 3600)

        print("\n📊 === STATISTIQUES DES TRANSACTIONS ===")
        print(f"🧾 Nombre de trades : {num_trades}")
        print(f"✅ Trades gagnants : {num_winning}")
        print(f"❌ Trades perdants : {num_losing}")
        print(f"🔁 Win rate : {win_rate:.2f}%")
        print(f"🧭 Profit Factor : {profit_factor:.2f}")
        print(f"📈 Gain moyen par trade : {avg_gain:.2f} $")
        print(f"📉 Perte moyenne par trade : {avg_loss:.2f} $")
        print(f"⏳ Durée moyenne d’un trade : {avg_duration_days:.2f} jours")
        
        equity_curve = pd.DataFrame({'Equity': results['_equity_curve']['Equity']}, index=df.index[-len(results['_equity_curve']):])
        plot_trading_results(df, equity_curve)
        
    except Exception as e:
        print(f"\n❌ ERREUR CRITIQUE: {str(e)}")
        print("Problèmes possibles:")
        print("- Chemins d'accès incorrects aux fichiers")
        print("- Données manquantes ou corrompues")
        print("- Modèle incompatible")
        print("- Paramètres de séquence inadaptés à vos données")

# === POINT D'ENTRÉE ===
if __name__ == '__main__':
    run_backtest()

📊 Chargement des données...
🔮 Chargement du modèle et génération des signaux...
Index 365 hors limites pour la séquence seq_57 (max = 364)
Index 370 hors limites pour la séquence seq_58 (max = 364)
Index 385 hors limites pour la séquence seq_61 (max = 364)
Index 395 hors limites pour la séquence seq_63 (max = 364)
Index 405 hors limites pour la séquence seq_65 (max = 364)
Index 415 hors limites pour la séquence seq_67 (max = 364)
Index 375 hors limites pour la séquence seq_59 (max = 364)
Index 380 hors limites pour la séquence seq_60 (max = 364)
Index 390 hors limites pour la séquence seq_62 (max = 364)
Index 400 hors limites pour la séquence seq_64 (max = 364)
Index 410 hors limites pour la séquence seq_66 (max = 364)

🧪 Lancement du backtest...



✅ Backtest terminé avec succès!
Return final: 53.99%
Nombre de trades: 15

📊 === STATISTIQUES DES TRANSACTIONS ===
🧾 Nombre de trades : 15
✅ Trades gagnants : 8
❌ Trades perdants : 7
🔁 Win rate : 53.33%
🧭 Profit Factor : 1.72
📈 Gain moyen par trade : 1717.27 $
📉 Perte moyenne par trade : -1139.38 $
⏳ Durée moyenne d’un trade : 12.87 jours
